In [61]:
import cv2
import numpy as np
from collections import deque

In [62]:
def zoom_at(img, zoom, coord=None):
    h, w, _ = [ zoom * i for i in img.shape ]
    
    if coord is None: cx, cy = w/2, h/2
    else: cx, cy = [ zoom*c for c in coord ]
    
    img = cv2.resize( img, (0, 0), fx=zoom, fy=zoom)
    img = img[ int(round(cy - h/zoom * .5)) : int(round(cy + h/zoom * .5)),
               int(round(cx - w/zoom * .5)) : int(round(cx + w/zoom * .5)),
               : ]
    
    return img

In [63]:
def wire_brightness(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Бинаризация изображения для выделения тонкой нити
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # В случае, если нить белая на темном фоне, инвертируем бинарное изображение
    white_pixels = cv2.countNonZero(binary)
    black_pixels = binary.size - white_pixels
    if white_pixels > black_pixels:
        binary = cv2.bitwise_not(binary)

    # Удаление шумов с помощью морфологических операций (опционально)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    binary_clean = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Поиск контуров нити
    contours, _ = cv2.findContours(binary_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Если нет контуров, выходим из программы
    if len(contours) == 0:
        return 0, None

    # Находим наибольший контур по площади (предполагаем, что это нить)
    largest_contour = max(contours, key=cv2.contourArea)

    # Создание маски для нити
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

    # Вычисляем среднюю яркость нити на сером оригинальном изображении, используя полученную маску
    mean_brightness = cv2.mean(gray, mask=mask)[0]
    return mean_brightness, mask

In [ ]:
from skimage.morphology import skeletonize
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

%matplotlib qt

class ThreadBrightnessAnalyzer:
    """Класс для анализа яркости нити произвольной формы"""
    
    def __init__(self):
        # Настройка интерактивного отображения
        # plt.ion()
        self.fig, self.ax = plt.subplots(figsize=(12, 6))
        self.brightness_line, = self.ax.plot([], [], '-', linewidth=2, color='red')
        self.ax.set_xlabel('Расстояние вдоль нити (пиксели)')
        self.ax.set_ylabel('Яркость')
        self.ax.set_ylim(0, 255)
        
        # Создание окон визуализации
        cv2.namedWindow('wire analysis', cv2.WINDOW_NORMAL)
    
    def analyze(self, image, thread_mask, thickness=5):
        """
        Анализирует изображение и строит график яркости вдоль нити
        
        Args:
            image: Исходное изображение (BGR или grayscale)
            thread_mask: Бинарная маска нити
            thickness: Толщина для измерения яркости вдоль нити
            
        Returns:
            Координаты центральной линии нити и значения яркости
        """        
        # Получаем скелет маски нити
        skeleton = self._get_skeleton(thread_mask)
        
        # Получаем упорядоченные точки центральной линии нити
        centerline_points = self._get_centerline_points(skeleton)
        
        # Аппроксимируем центральную линию сплайном
        smoothed_points = self._smooth_centerline(centerline_points)
        
        # Если недостаточно точек, выходим
        if smoothed_points is None or len(smoothed_points[0]) < 2:
            print("Не удалось определить центральную линию нити")
            return None, None
        
        # Измеряем яркость вдоль аппроксимированной центральной линии
        distances, brightness_values = self._measure_brightness(
            image, smoothed_points, thickness)
        
        # Визуализируем результаты
        self._visualize_results(
            image, smoothed_points, distances, brightness_values)
        
        return smoothed_points, brightness_values
    def _get_skeleton(self, mask):
        """Получает скелет маски нити"""
        binary_mask = mask.astype(bool)
        skeleton = skeletonize(binary_mask)
        return skeleton.astype(np.uint8)
    
    def _get_centerline_points(self, skeleton):
        """Извлекает и упорядочивает точки центральной линии"""
        # Находим точки скелета
        y_coords, x_coords = np.where(skeleton > 0)
        
        if len(x_coords) == 0:
            return np.array([])
            
        # Объединяем координаты
        points = np.column_stack((x_coords, y_coords))
        
        # Находим две наиболее удаленные точки (концы нити)
        max_dist = 0
        endpoints = None
        
        # Если много точек, выбираем подмножество для ускорения
        if len(points) > 100:
            subset = points[np.random.choice(len(points), 100, replace=False)]
        else:
            subset = points
            
        for i in range(len(subset)):
            for j in range(i+1, len(subset)):
                dist = np.sum((subset[i] - subset[j])**2)
                if dist > max_dist:
                    max_dist = dist
                    endpoints = (subset[i], subset[j])
        
        if endpoints is None:
            return points
            
        # Ищем путь между концами нити
        ordered_points = self._find_path_between_endpoints(skeleton, endpoints)
        
        return np.array(ordered_points)
    
    def _find_path_between_endpoints(self, skeleton, endpoints):
        """Находит путь между двумя концами нити на скелете"""
        start, end = endpoints
        start = tuple(start)
        end = tuple(end)
        
        # Создаем копию скелета для поиска пути
        visited = np.zeros_like(skeleton, dtype=bool)
        queue = [(start, [start])]
        visited[start[1], start[0]] = True
        
        # Смещения для 8-связных соседей
        neighbors = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
        
        while queue:
            (x, y), path = queue.pop(0)
            
            # Если дошли до конечной точки
            if (x, y) == end:
                return path
                
            # Проверяем соседей
            for dx, dy in neighbors:
                nx, ny = x + dx, y + dy
                
                # Проверяем границы изображения
                if nx < 0 or ny < 0 or nx >= skeleton.shape[1] or ny >= skeleton.shape[0]:
                    continue
                    
                # Если точка на скелете и еще не посещена
                if skeleton[ny, nx] > 0 and not visited[ny, nx]:
                    visited[ny, nx] = True
                    queue.append(((nx, ny), path + [(nx, ny)]))
        
        # Если путь не найден, возвращаем все точки скелета
        y_coords, x_coords = np.where(skeleton > 0)
        return np.column_stack((x_coords, y_coords))
    
    def _smooth_centerline(self, points, smoothing=50):
        """Аппроксимирует центральную линию нити сплайном"""
        if len(points) < 4:
            return None
            
        # Разделяем x и y координаты
        x = points[:, 0]
        y = points[:, 1]
        
        # Используем сплайн для сглаживания
        try:
            tck, u = splprep([x, y], s=smoothing)
            # Генерируем равномерно распределенные точки вдоль сплайна
            u_new = np.linspace(0, 1, 200)
            x_new, y_new = splev(u_new, tck)
            return np.array([x_new, y_new])
        except Exception as e:
            print(f"Ошибка при сглаживании: {e}")
            # Если не удалось аппроксимировать сплайном, используем полином
            try:
                # Сортируем точки по x-координате
                sorted_idx = np.argsort(x)
                x_sorted = x[sorted_idx]
                y_sorted = y[sorted_idx]
                
                # Аппроксимируем полиномом
                degree = min(5, len(x) - 1)
                coeffs = np.polyfit(x_sorted, y_sorted, degree)
                poly = np.poly1d(coeffs)
                
                # Генерируем равномерно распределенные точки
                x_new = np.linspace(min(x), max(x), 200)
                y_new = poly(x_new)
                return np.array([x_new, y_new])
            except Exception as e:
                print(f"Ошибка при аппроксимации полиномом: {e}")
                return None
    
    def _measure_brightness(self, image, smoothed_points, thickness):
        """Измеряет яркость изображения вдоль нити"""
        # Конвертируем в grayscale, если изображение цветное
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        x_points, y_points = smoothed_points
        brightness = np.zeros(len(x_points))
        
        # Вычисляем расстояние вдоль нити
        dx = np.diff(x_points)
        dy = np.diff(y_points)
        segment_lengths = np.sqrt(dx**2 + dy**2)
        distances = np.zeros_like(x_points)
        distances[1:] = np.cumsum(segment_lengths)
        
        # Для каждой точки центральной линии
        for i in range(len(x_points)):
            x, y = int(round(x_points[i])), int(round(y_points[i]))
            
            # Проверяем границы изображения
            if 0 <= x < gray.shape[1] and 0 <= y < gray.shape[0]:
                # Определяем область вокруг точки
                x_min = max(0, x - thickness // 2)
                x_max = min(gray.shape[1], x + thickness // 2 + 1)
                y_min = max(0, y - thickness // 2)
                y_max = min(gray.shape[0], y + thickness // 2 + 1)
                
                # Извлекаем область и измеряем среднюю яркость
                region = gray[y_min:y_max, x_min:x_max]
                if region.size > 0:
                    brightness[i] = np.mean(region)
        
        return distances, brightness
    
    def _visualize_results(self, image, smoothed_points, distances, brightness):
        """Визуализирует результаты анализа"""
        # Визуализируем центральную линию на изображении
        vis_image = image.copy()
        x_points, y_points = smoothed_points
        
        # Рисуем центральную линию
        points = np.column_stack((x_points.astype(np.int32), y_points.astype(np.int32)))
        for i in range(1, len(points)):
            cv2.line(vis_image, tuple(points[i-1]), tuple(points[i]), (0, 255, 0), 1)
        
        # Добавляем точки для наглядности
        for i in range(0, len(points), 10):
            cv2.circle(vis_image, tuple(points[i]), 3, (0, 0, 255), -1)
        
        # Отображаем визуализацию
        cv2.imshow('wire analysis', vis_image)
        
        # Обновляем график
        self.brightness_line.set_data(distances, savgol_filter(brightness, 15, 5))
        self.ax.set_xlim(0, max(distances))
        self.fig.canvas.draw_idle()
        self.fig.canvas.flush_events()
    
    def close(self):
        """Закрывает все окна и освобождает ресурсы"""
        cv2.destroyAllWindows()
        plt.ioff()
        plt.close(self.fig)

In [ ]:
# Глобальные переменные для отслеживания состояния
drawing = False  # Индикатор того, что мы сейчас рисуем
start_x, start_y = -1, -1  # Начальная точка
end_x, end_y = -1, -1  # Конечная точка
roi_selected = False  # Флаг выбранной области

def mouse_callback(event, x, y, flags, param):
    global drawing, start_x, start_y, end_x, end_y, roi_selected
    
    # Начало выделения области - нажатие левой кнопки мыши
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
        end_x, end_y = x, y
        roi_selected = False
    
    # Отслеживание перемещения мыши во время выделения
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            end_x, end_y = x, y
    
    # Завершение выделения - отпускание кнопки мыши
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_x, end_y = x, y
        roi_selected = True

def main():
    global drawing, start_x, start_y, end_x, end_y, roi_selected
    
    # Инициализация захвата видео с камеры
    cap = cv2.VideoCapture(0)
    bri_deque = deque([0], maxlen=20)
    
    # Создание окна и регистрация обработчика событий мыши
    cv2.namedWindow('Video')
    cv2.setMouseCallback('Video', mouse_callback)

    cv2.namedWindow('Mask')

    analyzer = ThreadBrightnessAnalyzer()
    
    while True:
        ret, frame = cap.read()
        # frame = zoom_at(frame, 2)
        if not ret:
            break
        
        # Создаем копию кадра для отображения
        display_frame = frame.copy()
        
        # Если в процессе выделения, рисуем текущий прямоугольник
        if drawing:
            cv2.rectangle(display_frame, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
        
        # Если область выделена, обрабатываем её
        if roi_selected:
            # Определяем координаты прямоугольника в правильном порядке
            x1, y1 = min(start_x, end_x), min(start_y, end_y)
            x2, y2 = max(start_x, end_x), max(start_y, end_y)
            
            # Проверяем, что область имеет размер
            if x1 != x2 and y1 != y2:
                # Выделяем выбранную область
                roi = frame[y1:y2, x1:x2]
                
                # Здесь ваша логика обработки ROI
                bri, mask = wire_brightness(roi)
                bri_deque.append(bri)
                cv2.imshow('Mask', cv2.bitwise_and(roi, roi, mask=mask))
                
                # Рисуем рамку выделенной области
                cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                key = cv2.waitKey(1) & 0xFF
                if key == ord('a'):
                    analyzer.analyze(roi, mask, thickness=2)
        
        # Показываем результат
        mean_bri = sum(bri_deque) / len(bri_deque)
        display_frame = cv2.putText(display_frame, f'bri: {mean_bri:.1f}', org=(10,30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255))
        cv2.imshow('Video', display_frame)
        
        # Проверка нажатия клавиш
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC - выход
            break
        elif key == ord('r'):  # R - сброс выделения
            roi_selected = False
    
    # Освобождаем ресурсы
    analyzer.close()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [ ]:
import random
plt.plot([random.random() for _ in range(10)])